In [ ]:
#| label: irFig6jn
from repo2data.repo2data import Repo2Data
import os 
import pickle
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
from plotly import tools

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
DATA_ROOT = dataset_path / "qmrlab-t1-book" / "t1-book-neurolibre"    
filename = DATA_ROOT / "01" / "figure_6.pkl"

with open(filename, 'rb') as f:
    TR_range, TI_lowres, TI_highres, T1_mean, T1_std, data_mean, data_std, data_noiseless = pickle.load(f)

config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        x = np.squeeze(np.asarray(TI_lowres[ii,:])),
        y = np.squeeze(np.asarray(data_mean[ii,:])),
        error_y=dict(
            type='data',
            color = ('rgb(22, 96, 167)'),
            array=np.squeeze(np.asarray(data_std[ii,:])),
            visible=True
        ),
        line = dict(
            color = ('rgb(22, 96, 167)'),
            dash = 'dot'),
        mode = 'markers',
        name = 'Monte Carlo simulated signal',
        text = 'Monte Carlo simulated signal',
        hoverinfo = 'x+y+text') for ii in range(len(TR_range))]

data1[28]['visible'] = True

data2 = [dict(
        visible = False,
        x = np.squeeze(np.asarray(TI_highres[ii,:])),
        y = np.squeeze(np.asarray(data_noiseless[ii,:])),
        line = dict(
            color = ('rgb(247, 152, 19)'),
            ),
        name = 'Noiseless signal',
        text = 'Noiseless signal',
        hoverinfo = 'x+y+text') for ii in range(len(TR_range))]

data2[28]['visible'] = True

data_meanT1 = [dict(
    visible = False,
    x = TR_range,
    y = T1_mean,
    name = 'Mean T<sub>1</sub> (s)',
    text = 'Mean T<sub>1</sub> (s)',
    hoverinfo = 'x+y+text',
    xaxis='x2',
    yaxis='y2') for ii in range(len(TR_range))]

data_meanT1[15]['visible'] = True

data_stdT1 = [dict(
    visible = False,
    x = TR_range,
    y = T1_std,
    line = dict(
        color = ('rgb(222, 22, 22)'),
        ),
    name = 'STD T<sub>1</sub> (s)',
    text = 'STD T<sub>1</sub> (s)',
    hoverinfo = 'x+y+text',
    xaxis='x2',
    yaxis='y3') for ii in range(len(TR_range))]

data_stdT1[28]['visible'] = True

data = data2 + data1 + data_meanT1 + data_stdT1

steps = []
for i in range(len(TR_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(TR_range[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 28,
    currentvalue = {"prefix": "TR value (ms): <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=540,
    height=540,
    margin = dict(
                t=0,
                r=25,
                b=100,
                l=75),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.17,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Signal (magnitude)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.76,
            y=0.77,
            showarrow=False,
            text='<b>TR (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.40,
            y=0.35,
            showarrow=False,
            text='<b>Mean T<sub>1</sub> (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.00,
            y=0.35,
            showarrow=False,
            text='<b>STD T<sub>1</sub> (ms)<b>',
            font=dict(
                family='Times New Roman',
                size=14
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, 5000],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        domain=[0.5, 0.90],
        anchor='y2',
        mirror = True,
        side='top',
        ticks='inside',
        showline=True,
    ),
    yaxis2=dict(
        autorange=False,
        range=[500, 1300],
        domain=[0.05, 0.65],
        anchor='x2',
        mirror = True,
        ticks='inside',
        showline=True,
    ),
    yaxis3=dict(
        autorange=False,
        range=[0, 190],
        domain=[0.05, 0.65],
        anchor='x2',
        overlaying='y2',
        side='right',
        ticks='inside',
    ),
    legend=dict(
        x=0.3,
        y=1.35,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders,
    plot_bgcolor='white'
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'ir_fig_6.html', config = config)
